In [1]:
# imports
import pandas as pd
import requests
df = pd.read_csv("vancouver_bike_stations.csv")
df.head()

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,extra
0,00fa94ad698dc4a9e4d708d6fd32f294,Chilco & Barclay,49.291909,-123.140713,2025-05-15T22:10:18.971140+00:00Z,12,6,"{'uid': '0092', 'renting': True, 'returning': ..."
1,012d3e06901cc222b1c2cf0a2ace3a29,St George & Broadway,49.262321,-123.093060,2025-05-15T22:10:19.278717+00:00Z,4,9,"{'uid': '0248', 'renting': True, 'returning': ..."
2,029a505bd4422a1afd127987757f71a6,Britannia Parking Lot,49.275882,-123.071865,2025-05-15T22:10:19.223420+00:00Z,1,10,"{'uid': '0232', 'renting': True, 'returning': ..."
3,0438114d2e6b96118de69bc9775bb21e,Morton & Denman,49.288030,-123.142135,2025-05-15T22:10:18.888188+00:00Z,8,17,"{'uid': '0031', 'renting': True, 'returning': ..."
4,0459b7e93703980b853cd65a9dc60596,Thornton & National,49.273777,-123.092723,2025-05-15T22:10:19.277211+00:00Z,11,3,"{'uid': '0233', 'renting': True, 'returning': ..."


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
# Set up Foursquare API key & headers
fsq_api_key = "fsq3qngNK3ZpApaAwjmlU/vq81pHyylliFDOfOSKTdlTht8="
headers = {
    "Accept":        "application/json",
    "Authorization": fsq_api_key
}

# Query Foursquare for each station's nearby restaurants
fsq_raw = []
for _, row in df.iterrows():
    lat, lon, station = row["latitude"], row["longitude"], row["name"]
    params = {
        "ll":     f"{lat},{lon}",
        "radius": 1000,
        "query":  "restaurant",
        "limit":  5
    }
    url      = "https://api.foursquare.com/v3/places/search"
    resp     = requests.get(url, headers=headers, params=params).json()
    fsq_raw.append((station, resp))

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [3]:
# Parse out the response
poi_list = []
for station, raw in fsq_raw:
    for place in raw.get("results", []):
        poi_list.append({
            "station_name": station,
            "place_name":   place.get("name"),
            "address":      place["location"].get("formatted_address"),
            "distance":     place.get("distance")
        })

Put your parsed results into a DataFrame

In [4]:
# Put results into a DataFrame
fsq_df = pd.DataFrame(poi_list)
fsq_df.head()

,station_name,place_name,address,distance
0,Chilco & Barclay,Kingyo Izakaya,"871 Denman St (btwn Haro & Barclay), Vancouver...",308
1,Chilco & Barclay,Peaked Pies,"975 Denman St, Vancouver BC V6G 2M3",299
2,Chilco & Barclay,Tavola,"1829 Robson St (at Denman St), Vancouver BC V6...",354
3,Chilco & Barclay,Ma Dang Goul,"847 Denman St (at Haro St.), Vancouver BC V6G 2L7",335
4,Chilco & Barclay,Joe's Grill,"948 Denman St (at Barclay St), Vancouver BC V6...",311


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [5]:
# Set up Yelp API key & headers
yelp_api_key = "7gg7whyAZwnsfKi7dKfxtzGIh75_QO-iqFUPlzrfdiLPWsKSbXmSiws0jcgRgjRKdlFJ8Y17oNUybRVjIaLlp-fyZOhK1TO5NGiNP8gKW2IRXsQzp2VBjSGdpPIbaHYx"
headers = {
    "Authorization": f"Bearer {yelp_api_key}"
}

# Query Yelp for each station's nearby restaurants
yelp_raw = []
for _, row in df.iterrows():
    lat, lon, station = row["latitude"], row["longitude"], row["name"]
    params = {
        "term":      "restaurant",
        "latitude":  lat,
        "longitude": lon,
        "radius":    1000,
        "limit":     5
    }
    url      = "https://api.yelp.com/v3/businesses/search"
    resp     = requests.get(url, headers=headers, params=params).json()
    yelp_raw.append((station, resp))

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [6]:
# Parse out the response
biz_list = []
for station, raw in yelp_raw:
    for b in raw.get("businesses", []):
        biz_list.append({
            "station_name": station,
            "place_name":   b.get("name"),
            "address":      b["location"].get("address1"),
            "rating":       b.get("rating")
        })

Put your parsed results into a DataFrame

In [7]:
# Put results into a DataFrame
yelp_df = pd.DataFrame(biz_list)
yelp_df.head()

,station_name,place_name,address,rating
0,Chilco & Barclay,Kingyo,871 Denman Street,4.4
1,Chilco & Barclay,Nook,781 Denman Street,4.2
2,Chilco & Barclay,Guu with Garlic,1698 Robson Street,4.3
3,Chilco & Barclay,Tavola,1829 Robson Street,4.4
4,Chilco & Barclay,Robba da Matti West End,1906 Haro St.,3.9


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

**Yelp vs. Foursquare Coverage**  
- Yelp returned `yelp_df.shape[0]` POIs in total.  
- Yelp includes a `rating` field; Foursquare includes `distance`.  
- Based on **number of results** and **presence of ratings**, Yelp appears more comprehensive for Vancouver.

Get the top 10 restaurants according to their rating

In [8]:
# Save to CSV
fsq_df.to_csv("foursquare_pois.csv", index=False)
yelp_df.to_csv("yelp_pois.csv", index=False)

top_10 = yelp_df.sort_values(by="rating", ascending=False).head(10)
top_10[["place_name", "rating", "address"]]


,place_name,rating,address
849,Mainichi,5.0,3667 West Broadway
231,Mainichi,5.0,3667 West Broadway
1084,Mainichi,5.0,3667 West Broadway
773,Mainichi,5.0,3667 West Broadway
369,Pho Khanh Express,5.0,987 E Hastings Street
943,Pho Khanh Express,5.0,987 E Hastings Street
941,Bite Me Handhelds,5.0,1205 East Hastings Street
283,Mainichi,5.0,3667 West Broadway
13,Magari by Oca,5.0,1260 Commercial Drive
952,Magari by Oca,5.0,1260 Commercial Drive
